In [1]:
# ============================================================
# 📦 SETUP
# ============================================================

!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow matplotlib pandas --quiet

import sys
sys.path.append("/kaggle/working/edm2")


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 9.87 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2


In [2]:
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

import sys
sys.path.append("/kaggle/working/edm2")
from training import networks_edm2
from training import training_loop


In [3]:
# ============================================================
# 📦 LOAD CONDITIONAL EDM2 MODEL (40 ATTR, KARRAS)
# ============================================================

import torch, pickle

MODEL_PKL = "/kaggle/input/network-snapshot-0001572-0-100-kr-40attr/pytorch/default/1/network-snapshot-0001572-0.100_kr_40attr.pkl"

device = torch.device("cuda")

with open(MODEL_PKL, "rb") as f:
    net = pickle.load(f)["ema"].to(device).eval()

print("✅ Model loaded")
print("Image resolution:", net.img_resolution)
print("Label dim:", net.label_dim)


✅ Model loaded
Image resolution: 64
Label dim: 40


In [4]:
# ============================================================
# 📦 LOAD CONDITIONAL EDM2 MODEL
# ============================================================

import pickle
import torch

MODEL_PKL = "/kaggle/input/network-snapshot-0001572-0-100-kr-40attr/pytorch/default/1/network-snapshot-0001572-0.100_kr_40attr.pkl"

device = torch.device("cuda")

with open(MODEL_PKL, "rb") as f:
    net = pickle.load(f)["ema"].to(device).eval()

print("✅ Conditional model loaded")
print("Condition dim:", net.label_dim)


✅ Conditional model loaded
Condition dim: 40


In [5]:
# ============================================================
# 🎲 PURE NOISE + CONDITION
# ============================================================

C = net.img_channels
H = W = net.img_resolution

# Pure noise
noise = torch.randn(1, C, H, W, device=device)

# Condition (1 × 40)
labels = torch.tensor(attr_vec, device=device).unsqueeze(0)

print("Noise shape:", noise.shape)
print("Labels shape:", labels.shape)


NameError: name 'attr_vec' is not defined

In [6]:
from pathlib import Path
import textwrap

SAMPLER_PATH = Path("/kaggle/working/edm2/edm_step_sampler.py")

SAMPLER_PATH.write_text(textwrap.dedent("""
import os
import torch
import numpy as np
import PIL.Image

def edm_sampler_with_steps(
    net,
    noise,
    labels=None,
    gnet=None,
    randn_like=None,
    num_steps=32,
    sigma_min=0.002,
    sigma_max=80,
    rho=7,
    schedule_name="karras",
    guidance=1,
    save_dir=None,
    dtype=torch.float32,
    **_ignored,
):
    assert save_dir is not None
    os.makedirs(save_dir, exist_ok=True)
    device = noise.device

    # ------------------ Karras rho ------------------
    step_indices = torch.arange(num_steps, device=device)
    sigmas = (
        sigma_max ** (1 / rho)
        + step_indices / (num_steps - 1)
        * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))
    ) ** rho

    sigmas = torch.cat([sigmas, torch.zeros_like(sigmas[:1])])

    def denoise(x, t):
        Dx = net(x, t, labels).to(dtype)
        if guidance == 1 or gnet is None:
            return Dx
        ref = gnet(x, t, labels).to(dtype)
        return ref.lerp(Dx, guidance)

    x = noise * sigmas[0]

    for i in range(num_steps):
        t_cur = sigmas[i]
        t_next = sigmas[i + 1]

        d = (x - denoise(x, t_cur)) / t_cur
        x = x + (t_next - t_cur) * d

        img = x[0].detach().cpu().clamp(-1, 1)
        img = ((img + 1) / 2 * 255).permute(1, 2, 0).numpy().astype(np.uint8)

        PIL.Image.fromarray(img).save(
            os.path.join(save_dir, f"step_{i:03d}.png")
        )

    return x
"""))

print("✅ Conditional-ready edm_step_sampler.py written")


✅ Conditional-ready edm_step_sampler.py written


In [7]:
# ============================================================
# 🧪 CONDITIONAL EDM SAMPLER WITH STEP SAVING
# ============================================================

import os
import PIL.Image

def edm_sampler_with_steps(
    net,
    noise,
    labels,
    save_dir,
    num_steps=32,
    sigma_min=0.002,
    sigma_max=80,
    rho=7,
):
    os.makedirs(save_dir, exist_ok=True)
    device = noise.device

    # Karras rho schedule
    i = torch.arange(num_steps, device=device)
    sigmas = (
        sigma_max ** (1 / rho)
        + i / (num_steps - 1)
        * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))
    ) ** rho
    sigmas = torch.cat([sigmas, torch.zeros_like(sigmas[:1])])

    def denoise(x, t):
        return net(x, t, labels)

    x = noise * sigmas[0]

    for i in range(num_steps):
        t_cur, t_next = sigmas[i], sigmas[i+1]
        d = (x - denoise(x, t_cur)) / t_cur
        x = x + (t_next - t_cur) * d

        img = x[0].detach().cpu().clamp(-1, 1)
        img = ((img + 1) / 2 * 255).permute(1, 2, 0).numpy().astype(np.uint8)
        PIL.Image.fromarray(img).save(f"{save_dir}/step_{i:03d}.png")

    return sigmas[:-1]


In [9]:
# ============================================================
# 🧩 DEFINE 3 CONDITIONAL EXPERIMENTS (TEST SET)
# ============================================================

import pandas as pd
import numpy as np

ATTR_CSV = "/kaggle/input/celeva-64x64-dataset/celeba64/list_attr_celeba.csv"

df = pd.read_csv(ATTR_CSV)
attr_cols = df.columns[1:41]

# CelebA test split (0-indexed)
TEST_START = 182637
TEST_END   = 202599

test_df = df.iloc[TEST_START:TEST_END].reset_index(drop=True)

# Convert {-1, +1} → {0, 1}
attr_matrix = (test_df[attr_cols].values > 0).astype(np.float32)
attr_counts = attr_matrix.sum(axis=1)

# ------------------------------------------------------------
# 1️⃣ Max +1 attributes
# ------------------------------------------------------------
idx_max = np.argmax(attr_counts)
attrs_max = attr_matrix[idx_max]

# ------------------------------------------------------------
# 2️⃣ Min +1 attributes
# ------------------------------------------------------------
idx_min = np.argmin(attr_counts)
attrs_min = attr_matrix[idx_min]

# ------------------------------------------------------------
# 3️⃣ Bald + Mustache
# ------------------------------------------------------------
mask = (test_df["Bald"] == 1) & (test_df["Mustache"] == 1)
idx_bm = np.where(mask)[0][0]
attrs_bm = (test_df.loc[idx_bm, attr_cols].values > 0).astype(np.float32)

# ------------------------------------------------------------
# FINAL EXPERIMENT DICT ✅
# ------------------------------------------------------------
EXPERIMENTS = {
    "max_attrs": attrs_max,
    "min_attrs": attrs_min,
    "bald_mustache": attrs_bm,
}

print("✅ Experiments ready:")
for k, v in EXPERIMENTS.items():
    print(f"{k:15s} → active attributes = {int(v.sum())}")


✅ Experiments ready:
max_attrs       → active attributes = 20
min_attrs       → active attributes = 1
bald_mustache   → active attributes = 15


In [10]:
# ============================================================
# 🚀 RUN 3 CONDITIONAL EXPERIMENTS
# ============================================================

C = net.img_channels
H = W = net.img_resolution

for name, attr_vec in EXPERIMENTS.items():
    print(f"\n🚀 Running experiment: {name}")

    # Pure noise
    noise = torch.randn(1, C, H, W, device=device)

    # Conditional attributes (1 × 40)
    labels = torch.tensor(attr_vec, device=device).unsqueeze(0)

    outdir = f"/kaggle/working/cond_{name}_karras_steps"

    sigmas = edm_sampler_with_steps(
        net=net,
        noise=noise,
        labels=labels,
        save_dir=outdir,
        num_steps=32,
    )

    print(f"✅ Saved steps to: {outdir}")

print("\n🎉 All conditional trajectories generated successfully")



🚀 Running experiment: max_attrs
✅ Saved steps to: /kaggle/working/cond_max_attrs_karras_steps

🚀 Running experiment: min_attrs
✅ Saved steps to: /kaggle/working/cond_min_attrs_karras_steps

🚀 Running experiment: bald_mustache
✅ Saved steps to: /kaggle/working/cond_bald_mustache_karras_steps

🎉 All conditional trajectories generated successfully


In [11]:
# ============================================================
# 🧩 STITCH CONDITIONAL TRAJECTORIES (4×8, σ-ANNOTATED)
# ============================================================

from PIL import Image, ImageDraw, ImageFont
import torch, os, math

# ------------------------------------------------------------
# Experiment folders
# ------------------------------------------------------------
EXPERIMENT_DIRS = {
    "bald_mustache": "/kaggle/working/cond_bald_mustache_karras_steps",
    "max_attrs": "/kaggle/working/cond_max_attrs_karras_steps",
    "min_attrs": "/kaggle/working/cond_min_attrs_karras_steps",
}

OUT_ROOT = "/kaggle/working"

# ------------------------------------------------------------
# Layout
# ------------------------------------------------------------
ROWS = 4
COLS = 8
TOP_PAD = 32
ROW_GAP = 18

# ------------------------------------------------------------
# Karras σ parameters (MUST match sampler)
# ------------------------------------------------------------
sigma_min = 0.002
sigma_max = 80
rho = 7
NUM_STEPS = ROWS * COLS

# ------------------------------------------------------------
# Compute Karras sigmas
# ------------------------------------------------------------
i = torch.arange(NUM_STEPS)
sigmas = (
    sigma_max ** (1 / rho)
    + i / (NUM_STEPS - 1)
    * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))
) ** rho

# ------------------------------------------------------------
# Safe font (Kaggle-compatible)
# ------------------------------------------------------------
try:
    font = ImageFont.truetype("DejaVuSans.ttf", 13)
except OSError:
    font = ImageFont.load_default()

# ------------------------------------------------------------
# Process each experiment
# ------------------------------------------------------------
for name, step_dir in EXPERIMENT_DIRS.items():
    print(f"🔧 Stitching: {name}")

    # Load step images
    imgs = [
        Image.open(os.path.join(step_dir, f"step_{i:03d}.png"))
        for i in range(NUM_STEPS)
    ]

    w, h = imgs[0].size

    canvas_w = COLS * w
    canvas_h = ROWS * (h + TOP_PAD + ROW_GAP)

    canvas = Image.new("RGB", (canvas_w, canvas_h), "white")
    draw = ImageDraw.Draw(canvas)

    # --------------------------------------------------------
    # Paste images + σ labels
    # --------------------------------------------------------
    for idx, img in enumerate(imgs):
        r = idx // COLS
        c = idx % COLS

        x = c * w
        y = r * (h + TOP_PAD + ROW_GAP)

        # σ label
        draw.text(
            (x + w // 2, y + 10),
            f"σ = {sigmas[idx]:.2f}",
            anchor="mm",
            fill="black",
            font=font,
        )

        # image
        canvas.paste(img, (x, y + TOP_PAD))

    # --------------------------------------------------------
    # Save
    # --------------------------------------------------------
    out_path = f"{OUT_ROOT}/{name}_karras_4x8_sigma.png"
    canvas.save(out_path)
    print(f"✅ Saved: {out_path}")

print("\n🎉 All stitched images generated successfully")


🔧 Stitching: bald_mustache
✅ Saved: /kaggle/working/bald_mustache_karras_4x8_sigma.png
🔧 Stitching: max_attrs
✅ Saved: /kaggle/working/max_attrs_karras_4x8_sigma.png
🔧 Stitching: min_attrs
✅ Saved: /kaggle/working/min_attrs_karras_4x8_sigma.png

🎉 All stitched images generated successfully


In [12]:
# ============================================================
# 🧾 PRINT ATTRIBUTES FOR 3 CONDITIONAL EXPERIMENTS (CelebA)
# ============================================================

import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Load CelebA attributes
# ------------------------------------------------------------
ATTR_CSV = "/kaggle/input/celeva-64x64-dataset/celeba64/list_attr_celeba.csv"
df = pd.read_csv(ATTR_CSV)

img_col = df.columns[0]
attr_cols = list(df.columns[1:41])  # 40 attributes

# ------------------------------------------------------------
# Test split (CelebA official)
# Rows 182638–202599 (1-indexed)
# ------------------------------------------------------------
TEST_START = 182637  # zero-indexed
TEST_END   = 202599

test_df = df.iloc[TEST_START:TEST_END].copy()

# Convert {-1, +1} → {0, 1}
attr_matrix = (test_df[attr_cols].values > 0).astype(np.int32)

# ------------------------------------------------------------
# 1️⃣ Max +1 attributes
# ------------------------------------------------------------
max_idx = np.argmax(attr_matrix.sum(axis=1))
max_row = test_df.iloc[max_idx]
max_attrs = [a for a, v in zip(attr_cols, attr_matrix[max_idx]) if v == 1]

# ------------------------------------------------------------
# 2️⃣ Min +1 attributes (max −1)
# ------------------------------------------------------------
min_idx = np.argmin(attr_matrix.sum(axis=1))
min_row = test_df.iloc[min_idx]
min_attrs = [a for a, v in zip(attr_cols, attr_matrix[min_idx]) if v == 1]

# ------------------------------------------------------------
# 3️⃣ Bald + Mustache = 1
# ------------------------------------------------------------
bald_i = attr_cols.index("Bald")
mustache_i = attr_cols.index("Mustache")

mask = (attr_matrix[:, bald_i] == 1) & (attr_matrix[:, mustache_i] == 1)
bm_idx = np.where(mask)[0][0]   # deterministic first match
bm_row = test_df.iloc[bm_idx]
bm_attrs = [a for a, v in zip(attr_cols, attr_matrix[bm_idx]) if v == 1]

# ------------------------------------------------------------
# Pretty print
# ------------------------------------------------------------
def print_block(title, row, attrs):
    print("\n" + "="*60)
    print(title)
    print("="*60)
    print("Image ID :", row[img_col])
    print("Num +1 attrs:", len(attrs))
    print("Attributes:")
    for a in attrs:
        print("  •", a)

print_block("🔺 MAX ATTRIBUTES EXPERIMENT", max_row, max_attrs)
print_block("🔻 MIN ATTRIBUTES EXPERIMENT", min_row, min_attrs)
print_block("🧔 BALD + MUSTACHE EXPERIMENT", bm_row, bm_attrs)



🔺 MAX ATTRIBUTES EXPERIMENT
Image ID : 184620.jpg
Num +1 attrs: 20
Attributes:
  • Arched_Eyebrows
  • Bags_Under_Eyes
  • Big_Lips
  • Big_Nose
  • Black_Hair
  • Chubby
  • Double_Chin
  • Heavy_Makeup
  • High_Cheekbones
  • Mouth_Slightly_Open
  • Narrow_Eyes
  • No_Beard
  • Receding_Hairline
  • Rosy_Cheeks
  • Smiling
  • Wavy_Hair
  • Wearing_Earrings
  • Wearing_Lipstick
  • Wearing_Necklace
  • Young

🔻 MIN ATTRIBUTES EXPERIMENT
Image ID : 183075.jpg
Num +1 attrs: 1
Attributes:
  • No_Beard

🧔 BALD + MUSTACHE EXPERIMENT
Image ID : 182797.jpg
Num +1 attrs: 15
Attributes:
  • 5_o_Clock_Shadow
  • Bald
  • Big_Lips
  • Big_Nose
  • Double_Chin
  • Eyeglasses
  • Goatee
  • High_Cheekbones
  • Male
  • Mouth_Slightly_Open
  • Mustache
  • Receding_Hairline
  • Smiling
  • Wearing_Necktie
  • Young
